In [5]:
import pandas as pd

# Step 1: Load the dataset
url = "D:\Intellihack_Weather_Forecasting/data/question4-stock-data.csv"  # Replace with the actual file name or full path
df = pd.read_csv(url)

# Step 2: Verify the dataset
print("First 5 rows of the dataset:")
print(df.head())

# Step 3: Check for missing values
print("\nMissing values:")
print(df.isnull().sum())

First 5 rows of the dataset:
   Unnamed: 0        Date  Adj Close     Close      High       Low      Open  \
0           0  1980-03-17   2.296798  3.291227  3.344743  3.291227  0.000000   
1           1  1980-03-18   2.306134  3.304606  3.358122  3.304606  0.000000   
2           2  1980-03-19   2.306134  3.304606  3.304606  3.304606  3.304606   
3           3  1980-03-20   2.306134  3.304606  3.358122  3.304606  0.000000   
4           4  1980-03-21   2.362154  3.384880  3.438396  3.384880  0.000000   

    Volume  
0  41109.0  
1   9343.0  
2      0.0  
3  10277.0  
4   8409.0  

Missing values:
Unnamed: 0      0
Date          110
Adj Close      93
Close         117
High           95
Low           127
Open          103
Volume        145
dtype: int64


<>:4: SyntaxWarning: invalid escape sequence '\I'
<>:4: SyntaxWarning: invalid escape sequence '\I'
C:\Users\thara\AppData\Local\Temp\ipykernel_9796\239436450.py:4: SyntaxWarning: invalid escape sequence '\I'
  url = "D:\Intellihack_Weather_Forecasting/data/question4-stock-data.csv"  # Replace with the actual file name or full path


In [6]:
# Check for missing values
print(df.isnull().sum())

Unnamed: 0      0
Date          110
Adj Close      93
Close         117
High           95
Low           127
Open          103
Volume        145
dtype: int64


In [7]:
# Summary statistics
print(df.describe())

         Unnamed: 0     Adj Close         Close          High           Low  \
count  11291.000000  11198.000000  11174.000000  11196.000000  11164.000000   
mean    5645.000000     63.609130     72.026945     72.503100     71.665079   
std     3259.575279     52.266247     51.259828     51.550735     51.011632   
min        0.000000      2.259452      3.237711      3.237711      3.237711   
25%     2822.500000     19.224636     27.500000     27.789255     27.536156   
50%     5645.000000     50.608900     66.035000     66.724998     65.418751   
75%     8467.500000    104.723621    114.297503    114.892500    113.639999   
max    11290.000000    254.770004    254.770004    255.229996    253.589996   

               Open        Volume  
count  11188.000000  1.114600e+04  
mean      67.999259  2.144157e+05  
std       55.834401  3.883662e+05  
min        0.000000  0.000000e+00  
25%        0.000000  1.350000e+04  
50%       66.065002  9.032350e+04  
75%      114.269997  2.915750e+05  


In [13]:
# Decompose the time series to observe trends and seasonality
from statsmodels.tsa.seasonal import seasonal_decompose

# Ensure the Date column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Decompose the time series
decomposition = seasonal_decompose(df['Close'], period=30)  # Assuming a monthly seasonality
decomposition.plot()
plt.show()

ModuleNotFoundError: No module named 'statsmodels'

In [17]:
# Create lag features for the past 5 days
for i in range(1, 6):
    df[f'Lag_{i}'] = df['Close'].shift(i)

# Drop rows with NaN values (due to lag features)
df.dropna(inplace=True)

In [18]:
# Create moving averages
df['MA_7'] = df['Close'].rolling(window=7).mean()  # 7-day moving average
df['MA_30'] = df['Close'].rolling(window=30).mean()  # 30-day moving average

# Drop rows with NaN values (due to moving averages)
df.dropna(inplace=True)

In [19]:
# Create moving averages
df['MA_7'] = df['Close'].rolling(window=7).mean()  # 7-day moving average
df['MA_30'] = df['Close'].rolling(window=30).mean()  # 30-day moving average

# Drop rows with NaN values (due to moving averages)
df.dropna(inplace=True)

In [20]:
from sklearn.model_selection import train_test_split

# Define features (X) and target (y)
X = df.drop(['Close'], axis=1)
y = df['Close']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


ModuleNotFoundError: No module named 'sklearn'

In [21]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Train an XGBoost model
model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: {rmse}")

ModuleNotFoundError: No module named 'xgboost'

In [22]:
# Calculate directional accuracy
directional_accuracy = np.mean(np.sign(y_pred) == np.sign(y_test))
print(f"Directional Accuracy: {directional_accuracy}")

NameError: name 'np' is not defined

In [23]:
# Simulate trading performance
df_test = X_test.copy()
df_test['Actual'] = y_test
df_test['Predicted'] = y_pred
df_test['Return'] = df_test['Actual'].pct_change()
df_test['Strategy'] = df_test['Predicted'].pct_change()

# Calculate cumulative returns
df_test['Cumulative_Actual'] = (1 + df_test['Return']).cumprod()
df_test['Cumulative_Strategy'] = (1 + df_test['Strategy']).cumprod()

# Plot cumulative returns
plt.figure(figsize=(10, 6))
plt.plot(df_test['Cumulative_Actual'], label='Actual Returns')
plt.plot(df_test['Cumulative_Strategy'], label='Strategy Returns')
plt.title('Simulated Trading Performance')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.show()

NameError: name 'X_test' is not defined

In [24]:
# Save predictions to a CSV file
df_test[['Actual', 'Predicted']].to_csv('stock_predictions.csv', index=False)

SyntaxError: invalid non-printable character U+00A0 (3663936872.py, line 2)